In [14]:
import pandas as pd

df = pd.read_csv('asset.csv')

df.head()

,pr_ref,po_ref,item,brand,type,serial_number,ghrs_id,badge_id,assignment_date,location,remarks,dept/project,delivery_date,status,service_tag,username,device_name,specifications
0,11716954.0,1360307,DESKTOP,NaN,HP Prodesk 400 G6 SFF,1CZ014053M,NaN,NaN,NaN,Main Office 1st Floor,NaN,HSES,2021-09-14,In Use,NaN,Yanto Supriyanto,SPMWS014053M,NaN
1,11716954.0,1360307,DESKTOP,NaN,HP Prodesk 400 G6 SFF,1CZ01403DH,121178,OPC000036,NaN,MWL Container Q-Wall 7,NaN,PPCO,2021-09-14,In Use,NaN,Fitriyanto Danny,SPMWS01403DH,NaN
2,11716954.0,1360307,DESKTOP,NaN,HP Prodesk 400 G6 SFF,1CZ0140528,131518,OPC000035,NaN,Main Office 1st Floor,NaN,HSES,2021-09-13,In Use,NaN,Aprilia Lusy Christiani S,SPMWS0140528,NaN
3,11716954.0,1360307,DESKTOP,NaN,HP Prodesk 400 G6 SFF,1CZ01403DZ,186227,186227,NaN,MWL Office 3,NaN,Quality,2021-09-06,In Use,NaN,Pakpahan Perengki,SPMWS01403DZ,NaN
4,11716954.0,1360307,DESKTOP,NaN,HP Prodesk 400 G6 SFF,1CZ014054J,949258,949258,NaN,Assembly Container Q-wall 4,NaN,Production,2021-09-06,In Use,NaN,Biwer Marek,SPMWS014054J,NaN


In [15]:
df['brand'].value_counts()

brand
DELL         644
LOGITECH     550
HP           526
Logitech     104
Samsung       79
Lenovo        20
NVIDIA        20
ViewSonic      6
Dell           1
Name: count, dtype: int64

In [16]:
df['brand'] = df['brand'].str.lower()

In [17]:
df['brand'].value_counts()

brand
logitech     654
dell         645
hp           526
samsung       79
lenovo        20
nvidia        20
viewsonic      6
Name: count, dtype: int64

In [18]:
df['item'].value_counts()

item
DESKTOP     1097
MONITOR      960
HEADSET      553
LAPTOP       531
DOCKING      237
EARPHONE     101
SSD           79
VGA CARD      20
Name: count, dtype: int64

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3578 entries, 0 to 3577
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   pr_ref           1670 non-null   object
 1   po_ref           2831 non-null   object
 2   item             3578 non-null   object
 3   brand            1950 non-null   object
 4   type             3577 non-null   object
 5   serial_number    3488 non-null   object
 6   ghrs_id          2092 non-null   object
 7   badge_id         1375 non-null   object
 8   assignment_date  1122 non-null   object
 9   location         2540 non-null   object
 10  remarks          2039 non-null   object
 11  dept/project     2344 non-null   object
 12  delivery_date    1012 non-null   object
 13  status           3578 non-null   object
 14  service_tag      126 non-null    object
 15  username         2202 non-null   object
 16  device_name      1450 non-null   object
 17  specifications   1379 non-null   

In [20]:
df = df[~df['item'].isin(['SSD', 'VGA CARD'])]
df['item'].value_counts()

item
DESKTOP     1097
MONITOR      960
HEADSET      553
LAPTOP       531
DOCKING      237
EARPHONE     101
Name: count, dtype: int64

In [21]:
df_valid = df[df['serial_number'].notna()].copy()
print(f"✅ Valid serial numbers: {len(df_valid)} rows")

✅ Valid serial numbers: 3409 rows


In [22]:
df_valid['serial_number'] = df_valid['serial_number'].astype(str).str.strip()

In [24]:
print("\n" + "=" * 60)
print("🔍 EKSTRAKSI FITUR DARI SERIAL NUMBER")
print("=" * 60)

def extract_sn_features(sn):
    """Extract features from serial number"""
    return {
        'length': len(sn),
        'digit_count': sum(c.isdigit() for c in sn),
        'letter_count': sum(c.isalpha() for c in sn),
        'special_char_count': sum(not c.isalnum() for c in sn),
        'first_3_chars': sn[:3].upper() if len(sn) >= 3 else sn.upper(),
        'last_3_chars': sn[-3:].upper() if len(sn) >= 3 else sn.upper(),
        'first_char': sn[0].upper() if len(sn) > 0 else '',
        'has_dash': '-' in sn,
        'has_underscore': '_' in sn,
        'is_numeric_only': sn.isdigit(),
        'is_alpha_only': sn.isalpha(),
        'digit_ratio': sum(c.isdigit() for c in sn) / len(sn) if len(sn) > 0 else 0
    }
    
# Apply feature extraction
sn_features = df_valid['serial_number'].apply(extract_sn_features)
features_df = pd.DataFrame(sn_features.tolist())

# Combine with original data
df_analysis = pd.concat([df_valid.reset_index(drop=True), features_df], axis=1)


🔍 EKSTRAKSI FITUR DARI SERIAL NUMBER


In [25]:
print("\n📈 Statistik Panjang Serial Number:")
print(df_analysis['length'].describe())

print("\n📊 Distribusi Panjang SN per Kategori Item:")
length_by_item = df_analysis.groupby('item')['length'].agg(['mean', 'median', 'std', 'min', 'max'])
print(length_by_item.round(2))


📈 Statistik Panjang Serial Number:
count    3409.000000
mean       12.686712
std         8.090767
min         1.000000
25%        10.000000
50%        10.000000
75%        12.000000
max        85.000000
Name: length, dtype: float64

📊 Distribusi Panjang SN per Kategori Item:
           mean  median    std  min  max
item                                    
DESKTOP    9.93    10.0   0.64    7   22
DOCKING   30.59    23.0  20.26    2   85
EARPHONE  12.00    12.0   0.00   12   12
HEADSET   12.00    12.0   0.00   12   12
LAPTOP     8.11     7.0   1.54    7   19
MONITOR   14.36    10.0   5.22    1   28


In [26]:
print("\n" + "=" * 60)
print("🔤 ANALISIS PREFIX (3 Karakter Pertama)")
print("=" * 60)

prefix_analysis = df_analysis.groupby(['item', 'first_3_chars']).size().reset_index(name='count')
prefix_analysis = prefix_analysis.sort_values(['item', 'count'], ascending=[True, False])

print("\n🏆 Top 5 Prefix per Kategori Item:")
for item in df_analysis['item'].unique():
    if pd.notna(item):
        top_prefix = prefix_analysis[prefix_analysis['item'] == item].head(5)
        print(f"\n{item}:")
        for _, row in top_prefix.iterrows():
            print(f"  {row['first_3_chars']}: {row['count']} unit")


🔤 ANALISIS PREFIX (3 Karakter Pertama)

🏆 Top 5 Prefix per Kategori Item:

DESKTOP:
  SGH: 524 unit
  1CZ: 483 unit
  4CE: 51 unit
  128: 5 unit
  512: 2 unit

DOCKING:
  CN0: 155 unit
  9 O: 5 unit
  1 O: 4 unit
  10 : 4 unit
  5 O: 4 unit

EARPHONE:
  203: 70 unit
  210: 30 unit

HEADSET:
  251: 100 unit
  244: 50 unit
  245: 50 unit
  241: 49 unit
  211: 48 unit

LAPTOP:
  5CG: 153 unit
  5CD: 29 unit
  CNU: 3 unit
  DXN: 3 unit
  HS7: 3 unit

MONITOR:
  CNC: 355 unit
  CN0: 299 unit
  TH0: 108 unit
  CNK: 71 unit
  6CM: 50 unit


In [27]:
print("\n" + "=" * 60)
print("🔢 ANALISIS KOMPOSISI KARAKTER")
print("=" * 60)

char_composition = df_analysis.groupby('item').agg({
    'digit_count': 'mean',
    'letter_count': 'mean',
    'special_char_count': 'mean',
    'digit_ratio': 'mean'
}).round(2)

print("\n📊 Rata-rata Komposisi per Kategori:")
print(char_composition)


🔢 ANALISIS KOMPOSISI KARAKTER

📊 Rata-rata Komposisi per Kategori:
          digit_count  letter_count  special_char_count  digit_ratio
item                                                                
DESKTOP          5.36          4.57                0.00         0.54
DOCKING         12.76         12.27                5.56         0.46
EARPHONE         7.79          4.21                0.00         0.65
HEADSET          7.36          4.64                0.00         0.61
LAPTOP           4.31          3.79                0.00         0.53
MONITOR          6.31          8.05                0.00         0.45


In [28]:
if 'brand' in df_analysis.columns:
    print("\n" + "=" * 60)
    print("🏭 ANALISIS PREFIX vs BRAND")
    print("=" * 60)
    
    brand_prefix = df_analysis.groupby(['brand', 'first_3_chars']).size().reset_index(name='count')
    brand_prefix = brand_prefix[brand_prefix['brand'].notna()]
    brand_prefix = brand_prefix.sort_values(['brand', 'count'], ascending=[True, False])
    
    print("\n🔍 Top 3 Prefix per Brand:")
    for brand in df_analysis['brand'].dropna().unique()[:10]:  # Top 10 brands
        top_brand_prefix = brand_prefix[brand_prefix['brand'] == brand].head(3)
        if len(top_brand_prefix) > 0:
            print(f"\n{brand}:")
            for _, row in top_brand_prefix.iterrows():
                print(f"  {row['first_3_chars']}: {row['count']} unit")


🏭 ANALISIS PREFIX vs BRAND

🔍 Top 3 Prefix per Brand:

dell:
  CN0: 454 unit
  TH0: 108 unit
  9 O: 5 unit

logitech:
  251: 100 unit
  203: 99 unit
  244: 50 unit

hp:
  CNC: 355 unit
  CNK: 71 unit
  6CM: 50 unit

lenovo:
  UMB: 20 unit

viewsonic:
  WYK: 6 unit


In [ ]:
print("\n" + "=" * 60)
print("📊 GENERATING VISUALIZATIONS...")
print("=" * 60)

fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Plot 1: Distribusi Panjang SN
axes[0, 0].hist(df_analysis['length'], bins=30, color='skyblue', edgecolor='black')
axes[0, 0].set_title('Distribusi Panjang Serial Number', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Panjang Serial Number')
axes[0, 0].set_ylabel('Frekuensi')
axes[0, 0].grid(axis='y', alpha=0.3)

# Plot 2: Panjang SN per Kategori Item
item_length = df_analysis.groupby('item')['length'].mean().sort_values()
axes[0, 1].barh(item_length.index, item_length.values, color='coral')
axes[0, 1].set_title('Rata-rata Panjang SN per Kategori', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Rata-rata Panjang')
axes[0, 1].grid(axis='x', alpha=0.3)

# Plot 3: Ratio Digit vs Letter
scatter_data = df_analysis.groupby('item').agg({
    'digit_ratio': 'mean',
    'letter_count': 'mean'
}).reset_index()
axes[1, 0].scatter(scatter_data['digit_ratio'], scatter_data['letter_count'], 
                   s=200, c=range(len(scatter_data)), cmap='viridis', alpha=0.6)
for idx, row in scatter_data.iterrows():
    axes[1, 0].annotate(row['item'], (row['digit_ratio'], row['letter_count']), 
                        fontsize=9, ha='center')
axes[1, 0].set_title('Digit Ratio vs Letter Count', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Digit Ratio (0-1)')
axes[1, 0].set_ylabel('Avg Letter Count')
axes[1, 0].grid(alpha=0.3)

# Plot 4: Top 10 Prefix Overall
top_prefixes = df_analysis['first_3_chars'].value_counts().head(10)
axes[1, 1].barh(range(len(top_prefixes)), top_prefixes.values, color='lightgreen')
axes[1, 1].set_yticks(range(len(top_prefixes)))
axes[1, 1].set_yticklabels(top_prefixes.index)
axes[1, 1].set_title('Top 10 Prefix Serial Number', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Jumlah')
axes[1, 1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.savefig('serial_number_analysis.png', dpi=300, bbox_inches='tight')
print("✅ Visualisasi disimpan sebagai 'serial_number_analysis.png'")

# ============================
# 9. EXPORT HASIL ANALISIS
# ============================
# Export feature-enriched dataset
df_analysis.to_csv('asset_with_sn_features.csv', index=False)
print("✅ Dataset dengan fitur tambahan disimpan sebagai 'asset_with_sn_features.csv'")

In [29]:
# Export summary statistics
with open('sn_analysis_summary.txt', 'w', encoding='utf-8') as f:
    f.write("=" * 60 + "\n")
    f.write("SUMMARY ANALISIS POLA SERIAL NUMBER\n")
    f.write("=" * 60 + "\n\n")
    f.write(f"Total records: {len(df_analysis)}\n")
    f.write(f"Unique serial numbers: {df_analysis['serial_number'].nunique()}\n\n")
    f.write("STATISTIK PANJANG SN:\n")
    f.write(str(df_analysis['length'].describe()) + "\n\n")
    f.write("PANJANG SN PER KATEGORI:\n")
    f.write(str(length_by_item) + "\n\n")
    f.write("KOMPOSISI KARAKTER:\n")
    f.write(str(char_composition) + "\n")

print("✅ Summary report disimpan sebagai 'sn_analysis_summary.txt'")

print("\n" + "=" * 60)
print("✅ ANALISIS SELESAI!")
print("=" * 60)
print("\nFile yang dihasilkan:")
print("  1. serial_number_analysis.png (visualisasi)")
print("  2. asset_with_sn_features.csv (dataset + features)")
print("  3. sn_analysis_summary.txt (summary report)")

✅ Summary report disimpan sebagai 'sn_analysis_summary.txt'

✅ ANALISIS SELESAI!

File yang dihasilkan:
  1. serial_number_analysis.png (visualisasi)
  2. asset_with_sn_features.csv (dataset + features)
  3. sn_analysis_summary.txt (summary report)
